# BLS Data API - Test Notebook

This notebook demonstrates how to access Bureau of Labor Statistics (BLS) Consumer Price Index data via the FastAPI web service.

## 🚀 Quick Start

1. **Server Setup**: Make sure the BLS API server is running:
   ```bash
   python api.py
   ```

2. **Client Setup**: Copy `bls_client.py` to your project directory

3. **Update API URL**: Change the `API_URL` variable below to match your server

## 📋 Requirements
- requests
- pandas
- matplotlib
- seaborn (optional)

In [1]:
# Install required packages (run once)
# !pip install requests pandas matplotlib seaborn

In [2]:
# Import libraries
import requests
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

📊 Libraries imported successfully!


In [3]:
# UPDATE THIS URL TO MATCH YOUR SERVER
API_URL = "http://localhost:8000"  # Change to your server's IP if needed
# API_URL = "http://192.168.1.100:8000"  # Example for network access

print(f"API URL: {API_URL}")
print(f"Test Date: 2025-06")
print(f"Current Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

🌐 API URL: http://localhost:8000
📅 Test Date: 2025-06
🕒 Current Time: 2025-07-24 14:14:45


## Method 1: Using the BLS Client Module (Recommended)

In [4]:
# Import the BLS client (make sure bls_client.py is in your project directory)
try:
    from bls_client import BLSClient
    print("BLS Client imported successfully")
except ImportError as e:
    print(f"Could not import BLS Client: {e}")
    print("Make sure 'bls_client.py' is in your project directory")
    # Alternative: define a simple client inline
    class BLSClient:
        def __init__(self, api_url):
            self.api_url = api_url.rstrip('/')
        
        def get_data(self, categories, date):
            response = requests.post(f"{self.api_url}/data", json={
                "categories": categories,
                "date": date
            })
            if response.status_code == 200:
                result = response.json()
                if result.get("success"):
                    return pd.DataFrame(result["data"])
            return None
    print("Using inline BLS Client")

✅ BLS Client imported successfully


In [5]:
# Create BLS API client
client = BLSClient(API_URL)

# The client will automatically test the connection when created

✅ Connected to BLS API at http://localhost:8000
   Data available: True


In [6]:
# Check API health
health_response = requests.get(f"{API_URL}/health")

if health_response.status_code == 200:
    health = health_response.json()
    print("API Health Status:")
    print(f"   Status: {health.get('status', 'unknown')}")
    print(f"   Data Available: {health.get('data_available', False)}")
    print(f"   Latest File: {health.get('latest_file', 'None')}")
    print(f"   Categories Count: {health.get('categories_count', 0)}")
else:
    print(f"API Health Check Failed: {health_response.status_code}")

🏥 API Health Status:
   Status: healthy
   Data Available: True
   Latest File: cpi-u-202506.xlsx
   Categories Count: 99


In [7]:
# Get available categories
try:
    categories = client.get_categories(15)
    print(f"\nAvailable Categories (first {len(categories)}):")
    for i, cat in enumerate(categories, 1):
        print(f"   {i:2d}. {cat}")
except:
    print("Could not retrieve categories")

📋 Retrieved 14 available categories

📋 Available Categories (first 14):
    1. All items
    2. Food
    3. Food at home
    4. Cereals and bakery products
    5. Cereals and cereal products
    6. Flour and prepared flour mixes
    7. Breakfast cereal(4)
    8. Rice, pasta, cornmeal
    9. Rice(4)(5)(6)
   10. Bakery products(4)
   11. Bread(4)(5)
   12. White bread(4)(6)
   13. Bread other than white(4)(6)
   14. Fresh biscuits, rolls, muffins(5)


## 📊 Basic Data Loading

In [8]:
# Define categories for analysis
analysis_categories = [
    "All items",
    "Food",
    "Energy",
    "Shelter",
    "Transportation",
    "Medical care"
]

target_date = "2025-06"

print(f"Loading data for {len(analysis_categories)} categories:")
for i, cat in enumerate(analysis_categories, 1):
    print(f"   {i}. {cat}")
print(f"\nTarget Date: {target_date}")

🎯 Loading data for 6 categories:
   1. All items
   2. Food
   3. Energy
   4. Shelter
   5. Transportation
   6. Medical care

📅 Target Date: 2025-06


In [9]:
# Load BLS data using the client
df = client.get_data(analysis_categories, target_date)

if df is not None:
    print(f"\nSuccessfully loaded data for {len(df)} categories")
    print(f"\nData Preview:")
    display(df[['category', 'nsa_previous_month', 'nsa_current_month', 'current_month', 'previous_month']].head())
else:
    print("Failed to load data")

✅ Successfully loaded data for 6 categories

✅ Successfully loaded data for 6 categories

📋 Data Preview:


,category,nsa_previous_month,nsa_current_month,current_month,previous_month
0,All items,321.465,322.561,2025-06,2025-05
1,Food,338.598,339.498,2025-06,2025-05
2,Energy,280.097,284.307,2025-06,2025-05
3,Shelter,414.689,415.455,2025-06,2025-05
4,Transportation,272.946,273.391,2025-06,2025-05
